In [25]:
import ee
ee.Authenticate()

Enter verification code:  4/1AdQt8qjMtcmMAqvcXsfgtJzCZyvv9fl55X1G_uazw4PWrs5r-uoLgSQdLa4



Successfully saved authorization token.


In [26]:
ee.Initialize()

In [111]:
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import io
from rasterstats import zonal_stats
import fiona
import rasterio.mask
import geemap
import glob
import boto3

In [41]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

In [52]:
# display all comuns in df
pd.set_option('display.max_columns', None)

# Read input data

In [28]:
out_dir = os.getcwd()

In [31]:
# read esa land cover
esa_land_cover = ee.ImageCollection('ESA/WorldCover/v100').first()

In [29]:
# get list of urbanshift cities
boundary_georef = pd.read_csv('https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv') 
# remove boundaries with no defined units of analysis
boundary_georef = boundary_georef.dropna(subset=['units_boundary_name']).reset_index(drop=True)
print(boundary_georef.to_string())

             geo_name   level aoi_boundary_name units_boundary_name
0         ARG-Mendoza  region         ADM3union                ADM3
1   ARG-Mar_del_Plata    city              ADM3                ADM4
2         ARG-Ushuaia    city              ADM4                ADM5
3           ARG-Salta  region         ADM2union                ADM3
4    ARG-Buenos_Aires  region         ADM2union                ADM2
5        BRA-Teresina    city         ADM4union                ADM4
6        BRA-Teresina  region         ADM2union                ADM2
7   BRA-Florianopolis    city         ADM4union                ADM4
8   BRA-Florianopolis  region         ADM2union                ADM2
9           BRA-Belem    city         ADM4union                ADM4
10          BRA-Belem  region         ADM2union                ADM2
11       CRI-San_Jose  region         ADM2union                ADM2
12         RWA-Kigali    city         ADM4union                ADM4
13       SLE-Freetown    city     ADM4cityunion 

# Compute zonal stat

In [32]:
for i in range(0, len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+boundary_id_aoi)
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
    
    # process aoi level ------
    print("\n boundary_id_aoi: "+boundary_id_aoi)
    # read boundaries
    boundary_path = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    # compute zonal sata and store locally
    lulc_stats = os.path.join(out_dir, 'data\\lulc_stats\\lulc_stats_'+boundary_id_aoi+'.csv')

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    geemap.zonal_statistics_by_group(
        esa_land_cover,
        boundary_geo_ee,
        lulc_stats,
        statistics_type='PERCENTAGE',
        denominator=1000000,
        decimal_places=2,
    )
    
    # process unit of analysis level ------
    print("\n boundary_id_unit: "+boundary_id_unit)
    # read boundaries
    boundary_path = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id_unit+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    # compute zonal sata and store locally
    lulc_stats = os.path.join(out_dir, 'data\\lulc_stats\\lulc_stats_'+boundary_id_unit+'.csv')

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    geemap.zonal_statistics_by_group(
        esa_land_cover,
        boundary_geo_ee,
        lulc_stats,
        statistics_type='PERCENTAGE',
        denominator=1000000,
        decimal_places=2,
    )

0

 geo_name: ARG-Mendoza-ADM3union

 boundary_id_aoi: ARG-Mendoza-ADM3union
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Saif.Shabou\OneDrive - World Resources Institute\Documents\UrbanShift\github\cities-urbanshift\baseline-indicators\biodiversity\notebooks\data\lulc_stats\lulc_stats_ARG-Mendoza-ADM3union.csv

 boundary_id_unit: ARG-Mendoza-ADM3
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Saif.Shabou\OneDrive - World Resources Institute\Documents\UrbanShift\github\cities-urbanshift\baseline-indicators\biodiversity\notebooks\data\lulc_stats\lulc_stats_ARG-Mendoza-ADM3.csv
1

 geo_name: ARG-Mendoza-ADM3union

 boundary_id_aoi: ARG-Mar_del_Plata-ADM3
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Saif.Shabou\OneDrive - World Resources Institute\Documents\UrbanShift\github\cities-urbanshift\baseline-indicators\biodiversity\notebooks\data\lulc_stats\lulc_stats_ARG-Mar_del_Plata-ADM3.csv

 bou

# Process output

In [95]:
# append city stats
path = out_dir + '\\data\\lulc_stats'
files= glob.glob(path + "/*.csv")

lulc_stats_cities = pd.DataFrame()
for f in files:
    lulc_stats_city = pd.read_csv(f)
    lulc_stats_cities = lulc_stats_cities.append(lulc_stats_city)

In [96]:
# pivot data
lulc_stats_cities = pd.melt(lulc_stats_cities, 
                            id_vars=['geo_id','geo_name','geo_parent_name','geo_level'], 
                            value_vars=['Class_10', 'Class_20', 'Class_30', 'Class_40', 'Class_50','Class_60','Class_70',
                                       'Class_80','Class_90','Class_95','Class_100'])


In [97]:
# rename columns
lulc_stats_cities = lulc_stats_cities.rename(columns={"variable": "lulc_class_code",
                                                     "value": "percent"})
# recode class levels
lulc_stats_cities['lulc_class_code'] = lulc_stats_cities['lulc_class_code'].str.replace('Class_', '')

In [103]:
# recoding class labels
mapper = {'10': 'Trees', 
          '20': 'Shrubland', 
          '30': 'Grassland',
          '40': 'Cropland',
          '50': 'Built-up',
          '60': 'Barren / sparse vegetation',
          '70': 'Snow and ice',
          '80': 'Open water',
          '90': 'Herbaceous wetland',
          '95': 'Mangroves',
          '100': 'Moss and lichen'}

lulc_stats_cities['lulc_class_label'] = lulc_stats_cities['lulc_class_code'].map(mapper).fillna(lulc_stats_cities['lulc_class_code'])

In [108]:
# replace na by 0
lulc_stats_cities['percent'] = lulc_stats_cities['percent'].fillna(0)

In [109]:
# add data source and year
lulc_stats_cities['data_source'] = 'ESA world cover'
lulc_stats_cities['year'] = '2020'

In [110]:
lulc_stats_cities

,geo_id,geo_name,geo_parent_name,geo_level,lulc_class_code,percent,lulc_class_label,data_source,year
0,ARG-Buenos_Aires_ADM-2_1,José C. Paz,ARG-Buenos_Aires,ADM-2,10,0.237981,Trees,ESA world cover,2020
1,ARG-Buenos_Aires_ADM-2_2,La Matanza,ARG-Buenos_Aires,ADM-2,10,0.128051,Trees,ESA world cover,2020
2,ARG-Buenos_Aires_ADM-2_3,Berazategui,ARG-Buenos_Aires,ADM-2,10,0.253611,Trees,ESA world cover,2020
3,ARG-Buenos_Aires_ADM-2_4,Vicente López,ARG-Buenos_Aires,ADM-2,10,0.128976,Trees,ESA world cover,2020
4,ARG-Buenos_Aires_ADM-2_5,Moreno,ARG-Buenos_Aires,ADM-2,10,0.396034,Trees,ESA world cover,2020
...,...,...,...,...,...,...,...,...,...
21335,SLE-Freetown_region_ADM-4_83,Hamilton,SLE-Freetown_region,ADM-4,100,0.000000,Moss and lichen,ESA world cover,2020
21336,SLE-Freetown_region_ADM-4_84,Kent,SLE-Freetown_region,ADM-4,100,0.000000,Moss and lichen,ESA world cover,2020
21337,SLE-Freetown_region_ADM-4_85,Sattia/Tombo,SLE-Freetown_region,ADM-4,100,0.000000,Moss and lichen,ESA world cover,2020
21338,SLE-Freetown_region_ADM-4_86,York,SLE-Freetown_region,ADM-4,100,0.000000,Moss and lichen,ESA world cover,2020


# Upload in aws

In [112]:
# connect to s3
aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [116]:
key_data = 'indicators/cities_lulc_stats.csv'
bucket_name = 'cities-urbanshift' 
lulc_stats_cities.to_csv(
    f"s3://{bucket_name}/{data_key}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [117]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')

# Remove local files

In [121]:
import shutil
import sys

In [122]:
try:
    shutil.rmtree(path)
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror))